In [40]:
from core import Cell, World, COLORS, Food
import numpy as np 
import time 
from collections import defaultdict

In [ ]:
import numpy as np 
import time 

""" 
=====================================================================================
Logic 
1. World is created with spaces.
2. Cell is created with world.
3. Cell is born in the world.
4. Cell dies in the world.
5. Cell can be born in the world.
"""

#set up the colors
WHITE = np.array([255, 255, 255])
YELLOW = np.array([255, 255, 0])
RED = np.array([255, 0, 0])
BLUE = np.array([0, 0, 255])
GREEN = np.array([0, 255, 0])
BLACK = np.array([0, 0, 0])
ORANGE = np.array([255, 128, 0])
PURPLE = np.array([128,0,128])

COLORS = [RED, BLUE, GREEN, ORANGE, PURPLE]




class World: 
    """
    World is where all the object lives and dies.
    It gives spaces and some other informations.
    You can set the size of the world.
    """
    def __init__(self, size: int):
        self.HEIGHT = size 
        self.WIDTH = size

        self.spaces: np.ndarray = np.zeros((self.HEIGHT, self.WIDTH), dtype=object)

        # self.lifes_ever: int = 0
        # self.lifes: list[Cell] = []

        # self.foods_ever: int = 0
        # self.foods: list[Food] = []

        self.matter: dict[str, any] = defaultdict(list)
        self.matter_count: dict[str, int] = defaultdict(int)


    def get_avialable_spaces(self) -> np.ndarray[np.ndarray[int], np.ndarray[int]]:
        """Search a free space and give it back."""
        return np.stack(np.where(self.spaces == 0), axis=1)


class Matter:
    """
    Proto type of life.
    Sub classes: Cell, Food
    """
    def __init__(self, world: World) -> None:
        self.world: World = world
        self.alive = False 
        self.current_location = None 
        self.color: tuple[int, int, int] = None
        self.age: int = 0 
        self.born_time: time.time = None 
        self.name: str = None 
        self.class_name: str = self.__class__.__qualname__

        # If there's free space, then it can born.
        available_space = self.get_space()
        if available_space is None:
            raise Exception("No available space")
        else:
            self.born(available_space)

    def get_space(self) -> None | np.ndarray:
        available_spaces = self.world.get_avialable_spaces()
        len_available_space = len(available_spaces)
        if len_available_space == 0:
            return None
        else:
            location = np.random.choice(len_available_space)
            return available_spaces[location]
        
    def born(self, available_space) -> None:
        self.world.spaces[available_space[0], available_space[1]] = self 
        self.world.matter[self.class_name].append(self) # for examples: 'Cell': [instacnes..]
        self.world.matter_count[self.class_name] += 1

        self.alive = True
        self.current_location = available_space
        self.born_time = time.time()
        self.name = f"{self.class_name}_{self.world.matter_count[self.class_name]}"
        self.color = COLORS[np.random.choice(len(COLORS))] 
    
    def die(self) -> None: 
        if self.alive: 
            row, col = self.current_location
            if self.world.spaces[row, col] == self:
                self.world.spaces[row, col] = 0
            
            self.alive = False
            self.color = None 
            self.current_location = None 

            if self in self.world.matter[self.class_name]:
                self.world.matter[self.class_name].remove(self)
        else:
            print(f"{self.type} is already dead")

class Cell(Matter): 
    """ 
    Cell is a living one. it's born with name, age, color, face(direction).
    1. born: born with a given world
    2. die: die itself
    3. move: move somewhere
    4. eat
       """
    def __init__(self, world: World) -> None:
        # Only for cell(mover).
        self.face: int[0, 1, 2, 3] = None 

        super().__init__(world)


        """ available_space = self.get_space()
        if available_space is None:
            raise Exception("No available space")
        else:
            self.born(available_space) """

    def born(self, available_space) -> None:
        super().born(available_space)
        # self.world.spaces[available_space[0], available_space[1]] = self 
        # self.world.lifes.append(self)
        # self.world.lifes_ever += 1 
        
        # self.world.matter[self.class_name].append(self)
        # self.world.matter["cellc_ever"]
        
        # self.alive = True
        # self.current_location = available_space
        # self.born_time = time.time()
        # self.name = f"Cell_{self.world.lifes_ever}"
        # self.color = COLORS[np.random.choice(len(COLORS))] 
        
        self.face = np.random.choice([0, 1, 2, 3])    # clock wise: front=0 -> right=1 -> back=2 -> left=3

    """ def die(self) -> None:
        if self.alive: 
            row, col = self.current_location
            if self.world.spaces[row, col] == self:
                self.world.spaces[row, col] = 0
            
            self.alive = False
            self.color = None 
            self.current_location = None 

            if self in self.world.lifes:
                self.world.lifes.remove(self)
        else:
            print("Cell is already dead") """

    """ def get_space(self) -> None | np.ndarray:
        available_spaces = self.world.get_avialable_spaces()
        len_available_space = len(available_spaces)
        if len_available_space == 0:
            return None
        else:
            location = np.random.choice(len_available_space)
            return available_spaces[location] """
            
    def __repr__(self):
        if self.alive:
            return f"{self.name}" 
        else: 
            return "Unborn Cell"

    # Aging system
    def aging(self) -> None:
        current_time = time.time()
        elapsed_time = np.round(current_time - self.born_time, 2)

        if self.alive and elapsed_time % 5:
            self.age += 1 
            self.color = self.color * 0.975

            if self.age > 100:
                self.die()
    
    def search_to_move(self):
        """
        Try to move with face(direction), get new location to move.
        There's some rules. It cant' move beyond the world map. So, the widht and height of the world.
        Secondly, it should have a logic to bump with other cells.
        """
        if self.face == 0:  # front
            new_location = self.current_location + np.array([-1, 0])
        elif self.face == 1:    # right
            new_location = self.current_location + np.array([0, 1])
        elif self.face == 2:    # back
            new_location = self.current_location + np.array([1, 0])
        elif self.face == 3:    # left
            new_location = self.current_location + np.array([0, -1]) 
        else:
            raise ValueError("Unknown face value. It must be {0, 1, 2, 3}")
        return new_location
        
    def ask_next_move(self):
        """ 
        1. Cell sends a location of next move to World.
        2. World respond to the cell, what's in the location it asked.
        """
        new_location = self.search_to_move()
        if 0 <= new_location[0] < self.world.HEIGHT and 0 <= new_location[1] < self.world.WIDTH:
            next_step = self.world.spaces[new_location[0], new_location[1]] 

            # print(f"next_step: {next_step}")

            # If it's empty then go. 0 means empty, so free to move
            if next_step == 0: 
                self.move(new_location)
            # If it's a food then eat it.
            elif isinstance(next_step, Food):
                self.eat(next_step)
            else:
                self.turn_face()
        else:
            self.turn_face()
            # print("Not available to move next")

    def move(self, new_location):
        # Clear old location
        row, col = self.current_location 
        if self.world.spaces[row, col] == self:
            self.world.spaces[row, col] = 0
        # self.world.spaces = np.where(self.world.spaces == self, 0, self.world.spaces)
        
        # Update world spaces info
        self.current_location = new_location
        self.world.spaces[new_location[0], new_location[1]] = self

    def turn_face(self) -> None:
        # random choice
        new_face = np.where(self.face != [0, 1, 2, 3])[0] # except its previous face
        self.face = np.random.choice(new_face)

    def eat(self, food) -> None:
        food.die()


class Food(Matter):
    """
    food for cell
    1. location
    2. state {alive, dead}
    """
    def __init__(self, world: World):
        super().__init__(world)
        # self.born()

    # def born(self):
    #     location = self.get_space()
    #     if location is None:
    #         raise Warning("No available space")
    #     else:
    #         self.world.foods_ever += 1 
    #         self.name = f"Food_{self.world.foods_ever}"
    #         self.born_time = time.time()

    #         self.world.spaces[location[0], location[1]] = self 
    #         self.current_location = location
    #         self.alive = True
    #         self.world.foods.append(self)
    
    """ def die(self):
        if self.alive: 
            location = self.current_location
            self.world.spaces[location[0], location[1]] = 0
            self.alive = False
            self.color = None 
            self.world.foods.remove(self)
        else:
            print(f"{self} is already dead") """
        
    def get_space(self):
        available_spaces = self.world.get_avialable_spaces()
        len_available_space = len(available_spaces)
        if len_available_space == 0:
            return None
        else:
            location = np.random.choice(len_available_space)
            return available_spaces[location]


    def __repr__(self):
        if self.alive:
            return f"{self.name}" 
        else: 
            return "Unborn Cell"
        

In [231]:
w = World(4)
c1 = Cell(w)


In [242]:
w.spaces

array([[0, 0, 0, 0],
       [0, Cell_2, 0, 0],
       [0, 0, 0, 0],
       [0, 0, Food_1, 0]], dtype=object)

In [243]:
w.matter 

defaultdict(list, {'Cell': [Cell_2], 'Food': [Food_1]})

In [234]:
c2 = Cell(w)

In [236]:
w.matter_count

defaultdict(int, {'Cell': 2})

In [237]:
c1.face

2

In [238]:
f1 = Food(w)

In [241]:
c1.die()

In [90]:
t = type(c3)
t 

__main__.Cell

In [68]:
t == Cell 

True

In [72]:
a = {}

In [73]:
a[str(t)] = c3.name 

In [89]:
a 

{"<class '__main__.Cell'>": 'Cell_3'}

In [97]:
c4 = Cell(w) 

In [101]:
c4.__class__.__qualname__

'Cell'

array([[0, Cell_4, 0, 0],
       [0, 0, Cell_5, 0],
       [Cell_1, 0, 0, 0],
       [Cell_3, 0, Cell_2, 0]], dtype=object)